1) Questão 1: Implemente uma estratégia de trade automático baseado em duas médias móveis e verifique o desempenho quanto ao Índice Sharpe. Utilize um período mínimo de 3 anos e os 5 (cinco) ativos escolhidos na primeira avaliação.

In [ ]:
import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

acoes = ['CMIG4','ELET3','ENBR3','USIM5','GOAU4']


def download_acao(ticker,data_inicio,data_fim):
    yf.pdr_override()       
    ticker = ticker + '.SA'
    return pdr.get_data_yahoo(ticker, 
                          start=data_inicio, 
                          end=data_fim)
CMIG4_df = pd.DataFrame()
ELET3_df = pd.DataFrame()
ENBR3_df = pd.DataFrame()
USIM5_df = pd.DataFrame()
GOAU4_df = pd.DataFrame()


In [ ]:
CMIG4_df = download_acao('CMIG4',datetime.datetime(2018,9,1),datetime.datetime(2019,9,1))

Questão 2: Implemente uma estratégia de trade automático baseado em duas médias móveis anterior, adicionando à aplicação o RSI ou MACD. Verifique o desempenho quanto ao Índice Sharpe nos mesmos ativos e tempo da questão 1.

Questão 3: Escolha uma estratégia de trade automático de contra tendência, apresentada na Referência Palex . Verifique o desempenho quanto ao Índice Sharpe nos mesmos ativos e tempo da questão 1.

Questão 4: Compare os resultados das três estrategias de trades, quanto: a) Retorno de Capital, b) Número de Trades, c) Taxa de Acertos, d) Fator de Lucro, e) Índice Sharpe e o f) retorno dos ativos avaliados na condição buy&hold.